## Semillero de Macroeconomía (2025 -II). Área de macroeconomía computacional.

De antemano, se agradece a los participantes del aréa de macroeconomía computacional del semestre pasado, pues éste trabjo se hizo con base en su notebook "Introducción al curso de macroeconomía en Python".

### Implementación de modelos macroeconómicos con datos reales:

Si bien resulta interesante ver cómo un modelo se comporta un modelo con datos teoricos, resulta aun más interesante como éste se comporta con datos reales. Así, éste modelo también será implementado con información empírica obtenida de la página oficial de  [Estadísticas del Banco de la república](https://suameca.banrep.gov.co/buscador-de-series/#/) . Esto permitirá contrastar el comportamiento esperado bajo supuestos teóricos con evidencia real, poniendo a prueba la validez y la aplicabilidad del marco intertemporal en un contexto concreto de la economía colombiana.

##### Modelo a correr (Obtenido del cuaderno "AFinal_Semillero_Macro")

$$ Max\ ln(C_1) + \beta \cdot \ln(C_2) \\s.a\\ C_2 = (1 + r_0) (Y_1 - C_1) + Y_2 $$

El consumidor elige cuánto consumir en cada periodo $(C_1)$ y $(C_2)$ para **maximizar su utilidad intertemporal**, donde $ \beta \in (0 ,  1) $ refleja cuánto valora el consumo futuro.

**Planteamiento del lagrangiano**

$$ ℒ = ln(C_1) + \beta \cdot \ln(C_2) +\lambda((1 + r_0) (Y_1 - C_1) + Y_2 - C_2 )$$

La restricción presupuestaria intertemporal ya está incorporada en el Lagrangiano mediante el multiplicador $\lambda$, lo que permite resolver el problema como una sola expresión unificada.


---

#### 1. Cargue y limpieza de datos:


Como la gran mayoria de los datos en internet se encuentran en forma no estructurada (i.e, poco organizada y entendible para proveer *insights* o *foresights*) entonces se debe hacer un correcto data-wrangling de la data. La data a organizar será:

    - Tasa de política monetaria (Dato fin de trimestre) – %

    - Producto Interno Bruto (PIB) nominal, Trimestral, metodología: 2015, Ajuste estacional (Dato fin de trimestre) – Miles de millones COP

    - Consumo final, real (Dato fin de trimestre) – Miles de millones COP

    - PTF, Inversión = Ahorro. Al tener un modelo cerrado éste supuesto es válido.

Así las cosas:
  

#### 2. Ejecución del modelo

In [95]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
from scipy.optimize import minimize

In [199]:
os.listdir(r"C:\Users\ZenBook Flip\Documents\Uniandes Cursos\Semillero Macro") #vemos la data que se encuentra en nuestro directorio.

['AFinal_Semillero_Macro - copia.ipynb',
 'AFinal_Semillero_Macro.ipynb',
 'anex-ProduccionConstantes-IItrim2025.xlsx',
 'graficador_series.xlsx',
 'ModelosDataReal V2.ipynb',
 'ModelosDataReal.ipynb']

In [ ]:
df = pd.read_excel("graficador_series.xlsx") #submos nuestros archivo donde esta la data.
df #vemos la data

,Fecha,Tasa de política monetaria(Dato fin de semestre),Tasa de política monetaria(Dato fin de trimestre),"Producto Interno Bruto (PIB) nominal, Trimestral, metodología: 2015, Ajuste estacional(Dato fin de trimestre)","Consumo final, real(Dato fin de trimestre)"
0,dd/mm/aaaa,%,%,Miles de millones COP,Miles de millones COP
1,31/03/1998,-,"30,00",-,-
2,30/06/1998,"32,00","32,00",-,-
3,30/09/1998,-,"32,00",-,-
4,31/12/1998,"26,00","26,00",-,-
...,...,...,...,...,...
109,31/03/2025,-,"9,50","451.477,58","235.099,68"
110,30/06/2025,"9,25","9,25","453.675,55","236.800,73"
111,NaN,NaN,NaN,NaN,NaN
112,NaN,NaN,NaN,NaN,NaN


In [201]:
df.columns #se ven las columnas de del DataFrame

Index(['Fecha', 'Tasa de política monetaria(Dato fin de semestre)',
       'Tasa de política monetaria(Dato fin de trimestre)',
       'Producto Interno Bruto (PIB) nominal, Trimestral, metodología: 2015, Ajuste estacional(Dato fin de trimestre)',
       'Consumo final, real(Dato fin de trimestre)'],
      dtype='object')

In [202]:
#El análisis solo se hará con los años 2023 en adelante:

df = df.drop(df.index[:101]) 
#con .drop(dfindex[:k]) se eliminan todas las filas del 2022 hacia atrás: k es "hasta el número" de index de fila que se quiere eliminar...
 #...y solo se dejan las filas con indíce k en adelante.
df

,Fecha,Tasa de política monetaria(Dato fin de semestre),Tasa de política monetaria(Dato fin de trimestre),"Producto Interno Bruto (PIB) nominal, Trimestral, metodología: 2015, Ajuste estacional(Dato fin de trimestre)","Consumo final, real(Dato fin de trimestre)"
101,31/03/2023,-,"13,00","394.442,31","226.302,56"
102,30/06/2023,"13,25","13,25","393.730,58","225.383,45"
103,30/09/2023,-,"13,25","396.048,37","226.009,67"
104,31/12/2023,"13,00","13,00","400.340,74","225.240,32"
105,31/03/2024,-,"12,25","409.316,56","226.187,36"
106,30/06/2024,"11,75","11,75","421.553,17","228.460,59"
107,30/09/2024,-,"10,75","429.896,10","228.647,27"
108,31/12/2024,"9,50","9,50","445.681,35","232.668,06"
109,31/03/2025,-,"9,50","451.477,58","235.099,68"
110,30/06/2025,"9,25","9,25","453.675,55","236.800,73"


In [203]:
#Se limpian strings: 
 #si hay columnas con celdas que posean strings y a la vez números se deben limpiar los strings para poder hacer análisis numéricos. Entonces:
df.drop(113, inplace = True)
df = df.drop(df.columns[1], axis=1)  #se elimina columna innecesaria
df

,Fecha,Tasa de política monetaria(Dato fin de trimestre),"Producto Interno Bruto (PIB) nominal, Trimestral, metodología: 2015, Ajuste estacional(Dato fin de trimestre)","Consumo final, real(Dato fin de trimestre)"
101,31/03/2023,"13,00","394.442,31","226.302,56"
102,30/06/2023,"13,25","393.730,58","225.383,45"
103,30/09/2023,"13,25","396.048,37","226.009,67"
104,31/12/2023,"13,00","400.340,74","225.240,32"
105,31/03/2024,"12,25","409.316,56","226.187,36"
106,30/06/2024,"11,75","421.553,17","228.460,59"
107,30/09/2024,"10,75","429.896,10","228.647,27"
108,31/12/2024,"9,50","445.681,35","232.668,06"
109,31/03/2025,"9,50","451.477,58","235.099,68"
110,30/06/2025,"9,25","453.675,55","236.800,73"


In [204]:
#Se renombran columnas:
df.rename(columns={'Tasa de política monetaria(Dato fin de trimestre)': "Tasa Politica Monetaria",
       'Producto Interno Bruto (PIB) nominal, Trimestral, metodología: 2015, Ajuste estacional(Dato fin de trimestre)' : "PIB real",
       'Consumo final, real(Dato fin de trimestre)' : "Consumo real",
       'Inflación total(Dato fin de mes)' : 'Inflación'}, inplace=True)
df

,Fecha,Tasa Politica Monetaria,PIB real,Consumo real
101,31/03/2023,"13,00","394.442,31","226.302,56"
102,30/06/2023,"13,25","393.730,58","225.383,45"
103,30/09/2023,"13,25","396.048,37","226.009,67"
104,31/12/2023,"13,00","400.340,74","225.240,32"
105,31/03/2024,"12,25","409.316,56","226.187,36"
106,30/06/2024,"11,75","421.553,17","228.460,59"
107,30/09/2024,"10,75","429.896,10","228.647,27"
108,31/12/2024,"9,50","445.681,35","232.668,06"
109,31/03/2025,"9,50","451.477,58","235.099,68"
110,30/06/2025,"9,25","453.675,55","236.800,73"


In [205]:
#Se resetea el índice (esto es el index de antes pasa a ser una nueva columna llamada "index" y se crea un nuevo index numérico desde 0):
df.reset_index(inplace=True)
df

,index,Fecha,Tasa Politica Monetaria,PIB real,Consumo real
0,101,31/03/2023,"13,00","394.442,31","226.302,56"
1,102,30/06/2023,"13,25","393.730,58","225.383,45"
2,103,30/09/2023,"13,25","396.048,37","226.009,67"
3,104,31/12/2023,"13,00","400.340,74","225.240,32"
4,105,31/03/2024,"12,25","409.316,56","226.187,36"
5,106,30/06/2024,"11,75","421.553,17","228.460,59"
6,107,30/09/2024,"10,75","429.896,10","228.647,27"
7,108,31/12/2024,"9,50","445.681,35","232.668,06"
8,109,31/03/2025,"9,50","451.477,58","235.099,68"
9,110,30/06/2025,"9,25","453.675,55","236.800,73"


In [206]:
#Se elimina la columna index creada al resetear el índice (no nos sirve para nada en este punto de la limpieza):
del df["index"]
df

,Fecha,Tasa Politica Monetaria,PIB real,Consumo real
0,31/03/2023,"13,00","394.442,31","226.302,56"
1,30/06/2023,"13,25","393.730,58","225.383,45"
2,30/09/2023,"13,25","396.048,37","226.009,67"
3,31/12/2023,"13,00","400.340,74","225.240,32"
4,31/03/2024,"12,25","409.316,56","226.187,36"
5,30/06/2024,"11,75","421.553,17","228.460,59"
6,30/09/2024,"10,75","429.896,10","228.647,27"
7,31/12/2024,"9,50","445.681,35","232.668,06"
8,31/03/2025,"9,50","451.477,58","235.099,68"
9,30/06/2025,"9,25","453.675,55","236.800,73"


In [207]:
#Se eliminan las dos últimas filas del dataframe pues solo son Nan:
df.drop(10 , inplace = True)
df.drop(11 , inplace = True)
df

,Fecha,Tasa Politica Monetaria,PIB real,Consumo real
0,31/03/2023,"13,00","394.442,31","226.302,56"
1,30/06/2023,"13,25","393.730,58","225.383,45"
2,30/09/2023,"13,25","396.048,37","226.009,67"
3,31/12/2023,"13,00","400.340,74","225.240,32"
4,31/03/2024,"12,25","409.316,56","226.187,36"
5,30/06/2024,"11,75","421.553,17","228.460,59"
6,30/09/2024,"10,75","429.896,10","228.647,27"
7,31/12/2024,"9,50","445.681,35","232.668,06"
8,31/03/2025,"9,50","451.477,58","235.099,68"
9,30/06/2025,"9,25","453.675,55","236.800,73"


 ##### Una gran aclaración: ¿Qué es un tipo de dato datetime64? ¿Por qué utilizarlo? ¿Cuándo utilizarlo? 

Un tipo de dato "datetime64" en python (pandas) es un **tipo de dato temporal** que usa pandas (basado en NumPy) para representar fechas y horas de manera eficiente.
Así las cosas, éste tipo de dato(s) Permiten manejar, transformar y analizar información temporal en **series de tiempo** (time series). 

Éste tipo de datos se utilizan de manera extensiva en finanzas cuantitativas y macroeconomía, en donde normalmente los valores (observaciones) estan indexados en tiempo. Pues normalmente, en éstas áreas, usamos datos como:

$$ Y_{t} , \space C_{t+1} , \space \pi_{t+s} $$

En otras palabras, convierte una cadena de texto como "30/06/2025" en un objeto de "tiempo real",  el cual resulta entendible por pandas y Python para hacer operaciones como diferencias, agrupaciones o filtros por fecha.

Algo interesante de este tipo de formato, respecto a otros formatos temporales es que puede ir desde tiempo muy agregados (Yearly) [YE] hasta datos temporales excesivamente pequeños: nano segundos [ns]. 

Al aplicar la función, pd.to_datetime(), por defecto, se deja todo en formato **%Y - %M - %D horas: minutos: segundos**. Eso es lo que haremos con nuestro DataSet.

In [166]:
copia = df.copy() #se crea una copia del dataframe para tener un respaldo de la data original
copia["Fecha"] = pd.to_datetime(copia["Fecha"], dayfirst=True)
copia

,Fecha,Tasa Politica Monetaria,PIB real,Consumo real
0,2023-03-31,"13,00","394.442,31","226.302,56"
1,2023-06-30,"13,25","393.730,58","225.383,45"
2,2023-09-30,"13,25","396.048,37","226.009,67"
3,2023-12-31,"13,00","400.340,74","225.240,32"
4,2024-03-31,"12,25","409.316,56","226.187,36"
5,2024-06-30,"11,75","421.553,17","228.460,59"
6,2024-09-30,"10,75","429.896,10","228.647,27"
7,2024-12-31,"9,50","445.681,35","232.668,06"
8,2025-03-31,"9,50","451.477,58","235.099,68"
9,2025-06-30,"9,25","453.675,55","236.800,73"


In [ ]:
copia["Tasa Politica Monetaria"] = copia["Tasa Politica Monetaria"].str.replace(",", ".", regex=False).astype(float)

copia["PIB real"] = (
    copia["PIB real"]
    .str.replace(".", "", regex=False)   # quitar puntos de miles
    .str.replace(",", ".", regex=False)  # cambiar coma decimal a punto
    .astype(float)                       # convertir a float
)

# Limpiar Consumo real
copia["Consumo real"] = (
    copia["Consumo real"]
    .str.replace(".", "", regex=False)
    .str.replace(",", ".", regex=False)
    .astype(float)
)

##### ¿Cuales son las ventajas del formato datetime64?

1. Permite ordenar cronológicamente los datos.

2. Habilita operaciones de filtrado temporal (por año, mes, día, hora).

3. Facilita el uso de funciones time-series como:

    - .resample() → reagrupar por frecuencia (mensual, anual, etc.)

    -  .rolling() → ventanas móviles (promedios, sumas móviles)

    - .shift() → desplazar valores en el tiempo (lags)

        En **pandas**, el método `.shift(k)` **desplaza los valores de una serie temporal** hacia adelante o hacia atrás **\( k \)** periodos en el eje del tiempo.

        Formalmente, si tenemos una serie temporal $ X_t $ indexada por tiempo $t$, el *shift* se define como:

        $$
        X_{t}^{(k)} = X_{t - k}
        $$

        donde:

        - `.shift(1)` → crea el **rezago de primer orden**
        $
        X_{t}^{(1)} = X_{t - 1}
        $

        - `.shift(2)` → rezago de dos periodos
        $
        X_{t}^{(2)} = X_{t - 2}
        $

        - `.shift(-1)` → adelanta un periodo
        $
        X_{t}^{(-1)} = X_{t + 1}
        $

        ##### 🧮 Ejemplo numérico

        Supongamos la siguiente serie:

        | t | $ X_t $ |
        |---|------------|
        | 1 | 10 |
        | 2 | 12 |
        | 3 | 15 |
        | 4 | 18 |

        El **rezago de un periodo** $ X_{t-1} = X_t^{(1)} $ sería:

        | $t$ | $ X_t$ | $ X_t^{(1)} = X_{t-1} $|
        |---|------------|--------------------------|
        | 1 | 10 | — |
        | 2 | 12 | 10 |
        | 3 | 15 | 12 |
        | 4 | 18 | 15 |



Un ejercio interesante con nuestro data set puede​ ser obtener la tasa de crecmiento del PIB:



$$
g_t = \frac{PIB_t - PIB_{t-1}}{PIB_{t-1}}
$$

O en porcentaje,

$$
g_t(\%) = \left( \frac{PIB_t - PIB_{t-1}}{PIB_{t-1}} \right) \times 100
$$



In [169]:
copia["Crecimiento PIB real (%)"] = round(((copia["PIB real"] - copia["PIB real"].shift(1)) / copia["PIB real"].shift(1)) * 100, 2)
copia #vemos el data set con la nueva columna

,Fecha,Tasa Politica Monetaria,PIB real,Consumo real,Crecimiento PIB real (%)
0,2023-03-31,13.00,394442.31,226302.56,NaN
1,2023-06-30,13.25,393730.58,225383.45,-0.18
2,2023-09-30,13.25,396048.37,226009.67,0.59
3,2023-12-31,13.00,400340.74,225240.32,1.08
4,2024-03-31,12.25,409316.56,226187.36,2.24
5,2024-06-30,11.75,421553.17,228460.59,2.99
6,2024-09-30,10.75,429896.10,228647.27,1.98
7,2024-12-31,9.50,445681.35,232668.06,3.67
8,2025-03-31,9.50,451477.58,235099.68,1.30
9,2025-06-30,9.25,453675.55,236800.73,0.49



 Otro método de interés es  `.resample()`. En **pandas** se utiliza para **cambiar la frecuencia temporal** de una serie de tiempo.  
Permite **agrupar datos por periodos de tiempo** (por ejemplo, de días a meses o de trimestres a años) y aplicar una **función de agregación** como `mean()`, `sum()`, `last()`, etc.

> Es el equivalente temporal de `groupby()`, pero aplicado a un índice de fechas (`DatetimeIndex`).

---

#### ⚙️ Sintaxis general

$$
\text{serie\_resample} = \text{serie.resample(freq, axis=0, label='right', closed='right').función()}
$$

**Ejemplo:**
```python
df["PIB real"].resample("Q").mean()  # Promedio trimestral
df["PIB real"].resample("A").last()  # Último valor de cada año


Así, resample acepta los siguentes parámetros para ordernar los datos agregagados de manera ordernada:

| Código | Frecuencia  | Descripción                             |
|:-------:|--------------|------------------------------------------|
| `A` o `YE` | Anual       | Fin de año (31 de diciembre)            |
| `Q`       | Trimestral  | Fin de trimestre                        |
| `2QE`      | Semestral   | Dos trimestres (Ene–Jun / Jul–Dic)     |
| `M`       | Mensual     | Fin de mes                              |
| `W`       | Semanal     | Fin de semana (domingo)                 |
| `D`       | Diaria      | Cada día                                |
| `H`       | Horaria     | Cada hora                               |
| `T` o `min` | Por minuto | Cada minuto                             |
| `S`       | Por segundo | Cada segundo                            |


#### Ejemplo:

In [ ]:
#Primero seteamos la columna "Fecha" como índice del DataFrame para facilitar análisis temporales (OJO Debe estar en datetime):
copia.set_index("Fecha", inplace=True)

In [ ]:
copia[["Tasa Politica Monetaria", "PIB real", "Consumo real"]].resample("2QE").last() #se pasa a semestral

,Tasa Politica Monetaria,PIB real,Consumo real
Fecha,,,
2023-03-31,13.00,394442.31,226302.56
2023-09-30,13.25,396048.37,226009.67
2024-03-31,12.25,409316.56,226187.36
2024-09-30,10.75,429896.10,228647.27
2025-03-31,9.50,451477.58,235099.68
2025-09-30,9.25,453675.55,236800.73


In [181]:
copia[["Tasa Politica Monetaria", "PIB real", "Consumo real"]].resample("YE").last() #se pasa a anual

,Tasa Politica Monetaria,PIB real,Consumo real
Fecha,,,
2023-12-31,13.00,400340.74,225240.32
2024-12-31,9.50,445681.35,232668.06
2025-12-31,9.25,453675.55,236800.73


En general, es deseable tener nuestros datos macroeconómicos, si estan indexados al tiempo, en este formato "datetime", pues el manejo de datos resulta mucho más fácil con éste.

### Siguiendo con la limpieza de datos...

In [208]:
df["Fecha"] = pd.to_datetime(df["Fecha"], dayfirst=True)
df

,Fecha,Tasa Politica Monetaria,PIB real,Consumo real
0,2023-03-31,"13,00","394.442,31","226.302,56"
1,2023-06-30,"13,25","393.730,58","225.383,45"
2,2023-09-30,"13,25","396.048,37","226.009,67"
3,2023-12-31,"13,00","400.340,74","225.240,32"
4,2024-03-31,"12,25","409.316,56","226.187,36"
5,2024-06-30,"11,75","421.553,17","228.460,59"
6,2024-09-30,"10,75","429.896,10","228.647,27"
7,2024-12-31,"9,50","445.681,35","232.668,06"
8,2025-03-31,"9,50","451.477,58","235.099,68"
9,2025-06-30,"9,25","453.675,55","236.800,73"


In [ ]:
df["Tasa Politica Monetaria"] = df["Tasa Politica Monetaria"].str.replace(",", ".", regex=False).astype(float)

df["PIB real"] = (
    df["PIB real"]
    .str.replace(".", "", regex=False)   # quitar puntos de miles
    .str.replace(",", ".", regex=False)  # cambiar coma decimal a punto
    .astype(float)                       # convertir a float
)

# Limpiar Consumo real
df["Consumo real"] = (
    df["Consumo real"]
    .str.replace(".", "", regex=False)
    .str.replace(",", ".", regex=False)
    .astype(float)
)
df

,Fecha,Tasa Politica Monetaria,PIB real,Consumo real
0,2023-03-31,13.00,394442.31,226302.56
1,2023-06-30,13.25,393730.58,225383.45
2,2023-09-30,13.25,396048.37,226009.67
3,2023-12-31,13.00,400340.74,225240.32
4,2024-03-31,12.25,409316.56,226187.36
5,2024-06-30,11.75,421553.17,228460.59
6,2024-09-30,10.75,429896.10,228647.27
7,2024-12-31,9.50,445681.35,232668.06
8,2025-03-31,9.50,451477.58,235099.68
9,2025-06-30,9.25,453675.55,236800.73


In [ ]:
#En este caso no se indexará el DataFrame por la columna Fecha y solo se dejará el año en dicha columna:
df["Fecha"]= df["Fecha"].dt.year #Se deja solo el año en la columna Fecha

In [213]:
df

,Fecha,Tasa Politica Monetaria,PIB real,Consumo real
0,2023,13.00,394442.31,226302.56
1,2023,13.25,393730.58,225383.45
2,2023,13.25,396048.37,226009.67
3,2023,13.00,400340.74,225240.32
4,2024,12.25,409316.56,226187.36
5,2024,11.75,421553.17,228460.59
6,2024,10.75,429896.10,228647.27
7,2024,9.50,445681.35,232668.06
8,2025,9.50,451477.58,235099.68
9,2025,9.25,453675.55,236800.73


In [ ]:
res = (
    df.groupby("Fecha")
      .agg(**{
          "PIB real (suma)": ("PIB real", "sum"),
          "Consumo real (suma)": ("Consumo real", "sum"),
          "Tasa prom. (%)": ("Tasa Politica Monetaria", "mean")
      })
)
res
markdown_table = res.to_markdown(index=True) # index=False to exclude DataFrame index
#print(markdown_table)

##### Con base en la anterior limpieza obtenemos la siguente tabla:


|   Fecha |   PIB real (suma) |   Consumo real (suma) |   Tasa real prom. (%) |
|--------:|------------------:|----------------------:|-----------------:|
|    2023 |        1584562.0  |                902936 |          3.1775  |
|    2024 |       1706447.18  |                915963 |          -1.265 |
|    2025 |       905153      |                471900 |            -  |


#### 2. Ejecución del modelo en python

In [ ]:
# Y1: Ingreso en período 2023 (t)
# Y2: Ingreso en período 2024 (t+1)
# C1: Consumo en periodo 2023 (t)
# r_0: Tasa de interés en 2023 (t)

# beta: Factor de descuento intertemporal
Y1 = res.iloc[0,0]
Y2 = res.iloc[1,0]
C1 = res.iloc[1,1]
r_0 = res.iloc[0,2]
beta = 0.95


    # Bloque 3: Definir la función de utilidad
    def utilidad_total(C1, Y1, Y2, r_0, beta):
        """
        Calcula la utilidad total para un nivel dado de consumo en período 1
        Utiliza función de utilidad logarítmica

        Parámetros:
        C1: Consumo en período 1
        Y1: Ingreso en período 1
        Y2: Ingreso en período 2
        r: Tasa de interés
        beta: Factor de descuento

        Retorna:
        Valor de utilidad total
        """
        # El ahorro en período 1 es ingreso menos consumo
        S = Y1 - C1

        # Consumo en período 2 viene del ahorro más intereses y el ingreso del período 2
        C2 = (1 + r_0) * S + Y2

        # Si el consumo es negativo, retorna utilidad muy baja (restricción - el motivo se explica a en la seccion de optimizacion a continuacion)
        if C2 < 0 or C1 < 0:
            return -np.inf

        # Utilidad total es la suma de utilidad presente y futura descontada
        U = np.log(C1) + beta * np.log(C2)
        return U

utilidad_total(C1,Y1,Y2,r_0,beta)

29.143368538014336

In [220]:
# Bloque 4: Definir la función para optimización
def negativo_utilidad_total(C1, Y1, Y2, r_0, beta):
    """
    Retorna el negativo de la utilidad total para usar con minimize
    """
    return -utilidad_total(C1, Y1, Y2, r_0, beta)

negativo_utilidad_total(C1, Y1, Y2, r_0, beta)

-29.143368538014336

In [221]:
# Bloque 5: Función para resolver el modelo
def resolver_modelo(Y1, Y2, r, beta):
    """
    Encuentra el consumo óptimo que maximiza la utilidad total

    Parámetros:
    Y1: Ingreso en período 1
    Y2: Ingreso en período 2
    r: Tasa de interés
    beta: Factor de descuento

    Retorna:
    C1_optimo: Consumo óptimo en período 1
    C2_optimo: Consumo óptimo en período 2
    S_optimo: Ahorro óptimo en período 1
    """
    # Usa como punto inicial la mitad del ingreso del período 1
    resultado = minimize(
        lambda C1: negativo_utilidad_total(C1, Y1, Y2, r_0, beta),
        x0=[Y1/2],
        method='Nelder-Mead'
    )

    C1_optimo = resultado.x[0]
    S_optimo = Y1 - C1_optimo
    C2_optimo = (1 + r) * S_optimo + Y2

    return C1_optimo, C2_optimo, S_optimo

resolver_modelo(Y1, Y2, r_0, beta)

(874549.9616052152, 11735367.222326335, 710012.0383947848)

Así, con base en en los datos del Banco de la República, el consumo óptimo en el período 1 es aproximadamente 874,549 de pesos, el consumo óptimo en en el período 2 es aproximadamente 11,735367 de pesos, y el ahorro óptimo en el período 1 es aproximadamente 710,012 de pesos.

Note que el modelo, entonces, predice cuánto será el consumo en el 2026 y el ahorro óptimo. Éstos valores se pondrán a luz de los efectivamente reportados por el DANE y el Bánco de la república a final del 2026 o principios del 2027. Sin embargo, se cree que los valores del modelo sesgados, dados los supuestos del modelo (Por ejemplo Economía cerrada, lo cual no se cumple en el caso de Colombia). 